In [1]:
import pandas as pd
from konlpy.tag import Okt
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [2]:
tm = pd.read_csv('text_media.csv')

In [3]:
tm.head()

,Title,DateTime,Author,Rank,Category,Text,Images,Views,Href
0,탈모 3년째,2023-09-01 13:01:06,뷔놈,5,Fuck탈모,21살부터 시작해서 3년지나고나서야 약을 먹습니다 모나드정 피나스테리드 처방 받았는...,NaN,74.0,https://daedamo.com/story/1871888?overlaps=1
1,처음 처방받아서 질문남겨요,2023-09-01 12:08:44,K27135528642303…,3,고민상담,처방전 16500원\r\n모나시아정 63000원인데 3개월에 이정도면 적당한 가격인...,NaN,91.0,https://daedamo.com/story/1871881?overlaps=1
2,하남 아워스튜디오하남미사점 두피문신 후기 입니다.,2023-09-01 11:53:37,모닝98,2,Fuck탈모,20대부터 정수리 부분 탈모가 시작되 20대 중후반 부터 흑채 뿌리기 시작했습니다....,NaN,112.0,https://daedamo.com/story/1871880?overlaps=1
3,여의도 탈모성지다녀온후기씁니다,2023-09-01 11:21:25,K29993214812309…,2,Fuck탈모,회사 스트레스로 아침저녁으로. 머리빠지는듯해서\r\n\r\n걱정이였는데\r\n\r\...,NaN,168.0,https://daedamo.com/story/1871873?overlaps=1
4,M자 모발이식 질문,2023-09-01 11:17:19,빛바랜머리,2,Fuck탈모,모발에 힘도 없고 가늘고 M자까지 있네요\r\n스트레스 받아서 수술을 받을까 생각중...,NaN,142.0,https://daedamo.com/story/1871872?overlaps=1


In [4]:
tm['title_text'] = tm['Title'] + ' ' + tm['Text']
tm['title_text'].fillna('',inplace=True)

In [166]:
len(tm)

54005

In [168]:
tm['title_text']

0        탈모 3년째 21살부터 시작해서 3년지나고나서야 약을 먹습니다 모나드정 피나스테리드...
1        처음 처방받아서 질문남겨요 처방전 16500원\r\n모나시아정 63000원인데 3개...
2        하남 아워스튜디오하남미사점 두피문신 후기 입니다. 20대부터 정수리 부분 탈모가 시...
3        여의도 탈모성지다녀온후기씁니다 회사 스트레스로 아침저녁으로. 머리빠지는듯해서\r\n...
4        M자 모발이식 질문 모발에 힘도 없고 가늘고 M자까지 있네요\r\n스트레스 받아서 ...
                               ...                        
54000    혹시 피나하구 항우울제 같이먹어두될까요?? 항우울제 프로작하구 프로페시아 같이 먹어...
54001    프로페시아 3개월치 이상 사시는분들 꼭봐주세요 안녕하세요 프로페시아 4년차 복용중에...
54002    m자 조짐있어보이나요? <3년전>\r\n\r\n\r\n<최근>\r\n\r\n\r\n...
54003    21살인데 모발이식 해도 될까요?.. 탈모가 19살때 부터 와서 좀 심합니다.\r\...
54004    앞머리 M자말고도 그냥 헤어라인에 미녹시딜 바르고있었는데 M자말고도 중앙부분에 잔털...
Name: title_text, Length: 54005, dtype: object

In [165]:
tokenizer  = Okt()

In [189]:
import re

In [199]:
pattern = re.compile(r'[\n\t]+')

def make_tokens(doc):
    tokens = []
    if doc and (type(doc) == str or not np.isnan(doc)):
        cleaned_doc = pattern.sub(' ', doc)
        phrase = tokenizer.pos(cleaned_doc,norm=True,stem=True)
        tokens = [word[0] for word in phrase if word[0] and (word[1] in ['Noun', 'Adjective', 'Verb', 'Adverb', 'VerbPrefix', 'Suffix']) and (word[0] not in ['하다','있다'])]
    return tokens

In [200]:
res = []
for t in tm['title_text']:
    res.append(make_tokens(t))
res

KeyboardInterrupt: 

In [188]:
len(res)

36

In [16]:
cv = CountVectorizer(tokenizer=make_tokens,max_df=0.9,min_df=2)

In [17]:
dtm = cv.fit_transform(tm['title_text'])

In [46]:
feature_names = cv.get_feature_names_out()

# Convert the DTM to a list of lists containing tokenized words
dtm_as_list_of_lists = []

for doc_idx in range(len(tm['title_text'])):
    doc_tokens = [feature_names[i] for i in range(len(feature_names)) if dtm[doc_idx, i] > 0]
    dtm_as_list_of_lists.append(doc_tokens)

KeyboardInterrupt: 

In [18]:
dtm

<54005x17205 sparse matrix of type '<class 'numpy.int64'>'
	with 2339798 stored elements in Compressed Sparse Row format>

In [25]:
lda2 = LatentDirichletAllocation(n_components=2,random_state=1)
lda3 = LatentDirichletAllocation(n_components=3,random_state=1)
lda5 = LatentDirichletAllocation(n_components=5,random_state=1)
lda10 = LatentDirichletAllocation(n_components=10,random_state=1)

In [37]:
lda2.fit(dtm)
# dictionary = corpora.Dictionary(tokens)
# corpus = [dictionary.doc2bow(text) for text in tokens]

LatentDirichletAllocation(n_components=2, random_state=1)

In [ ]:
lda3.fit(dtm)

In [26]:
lda5.fit(dtm)

LatentDirichletAllocation(n_components=5, random_state=1)

In [ ]:
lda10.fit(dtm)

LatentDirichletAllocation(random_state=1)

In [28]:
len(cv.get_feature_names_out())

17205

In [31]:
tokenizer.pos('페',norm=True,stem=True)

[('페', 'Noun')]

In [19]:
cv.get_feature_names_out()[:1000]

array(['가', '가가', '가감', '가게', '가격', '가격표', '가경동', '가계', '가계도', '가공', '가과',
       '가관', '가구', '가글', '가글이', '가급적', '가기', '가까스로', '가까워지다', '가까이',
       '가깝다', '가꾸다', '가끔', '가끔가다', '가끔가다가', '가나', '가난', '가난하다', '가내',
       '가늘다', '가늘어지다', '가늠', '가늡니', '가능', '가능성', '가능하다', '가닌깐', '가다',
       '가다가', '가다듬다', '가닥', '가닥가닥', '가도', '가독성', '가두다', '가드', '가득',
       '가득하다', '가든', '가디', '가뜩이나', '가라', '가라앉다', '가라앉히다', '가락', '가랑비',
       '가래', '가량', '가려움', '가려움증', '가려웠', '가려웠구', '가려지다', '가려진', '가렵',
       '가렵거', '가렵거든', '가렵고', '가렵기', '가렵기도', '가렵기만', '가렵나', '가렵네', '가렵다',
       '가렵더', '가렵더군', '가렵던', '가렵습니', '가렵지', '가렵진', '가령', '가로', '가로수길',
       '가루', '가루약', '가르다', '가르마', '가르치다', '가르침', '가름', '가리다', '가리키다',
       '가림', '가마', '가만', '가만두다', '가만있다', '가만히', '가망', '가면', '가모', '가문',
       '가물', '가물가물', '가물다', '가뭄', '가밍', '가바', '가발', '가방', '가버리다', '가벼워지다',
       '가볍다', '가보', '가보다', '가봣는데', '가분', '가비', '가빠지다', '가뿐하다', '가쁘', '가사',
       '가산', '가상', '가설', '가성', '가세', '가소롭다', '가속', '가속도', '가수', '가수원',


In [35]:
top_words = lda5.components_[4].argsort()[-20:]
top_words

array([16952, 13207, 15005,  6984,  3870,   994, 13246,  6433,   461,
        1131, 12794,  7631,  3701,  8473,  5942, 12175,  6400,  5175,
       11806, 16054], dtype=int64)

In [51]:
import gensim.corpora as corpora
import gensim
from pprint import pprint

In [85]:
data_words = list(cv.get_feature_names_out())
stop_words = ['하다','있다']
data_words
def remove_stopwords(texts):
    return [word for word in texts if word not in stop_words]
data_words = remove_stopwords(data_words)
data_words


['가',
 '가가',
 '가감',
 '가게',
 '가격',
 '가격표',
 '가경동',
 '가계',
 '가계도',
 '가공',
 '가과',
 '가관',
 '가구',
 '가글',
 '가글이',
 '가급적',
 '가기',
 '가까스로',
 '가까워지다',
 '가까이',
 '가깝다',
 '가꾸다',
 '가끔',
 '가끔가다',
 '가끔가다가',
 '가나',
 '가난',
 '가난하다',
 '가내',
 '가늘다',
 '가늘어지다',
 '가늠',
 '가늡니',
 '가능',
 '가능성',
 '가능하다',
 '가닌깐',
 '가다',
 '가다가',
 '가다듬다',
 '가닥',
 '가닥가닥',
 '가도',
 '가독성',
 '가두다',
 '가드',
 '가득',
 '가득하다',
 '가든',
 '가디',
 '가뜩이나',
 '가라',
 '가라앉다',
 '가라앉히다',
 '가락',
 '가랑비',
 '가래',
 '가량',
 '가려움',
 '가려움증',
 '가려웠',
 '가려웠구',
 '가려지다',
 '가려진',
 '가렵',
 '가렵거',
 '가렵거든',
 '가렵고',
 '가렵기',
 '가렵기도',
 '가렵기만',
 '가렵나',
 '가렵네',
 '가렵다',
 '가렵더',
 '가렵더군',
 '가렵던',
 '가렵습니',
 '가렵지',
 '가렵진',
 '가령',
 '가로',
 '가로수길',
 '가루',
 '가루약',
 '가르다',
 '가르마',
 '가르치다',
 '가르침',
 '가름',
 '가리다',
 '가리키다',
 '가림',
 '가마',
 '가만',
 '가만두다',
 '가만있다',
 '가만히',
 '가망',
 '가면',
 '가모',
 '가문',
 '가물',
 '가물가물',
 '가물다',
 '가뭄',
 '가밍',
 '가바',
 '가발',
 '가방',
 '가버리다',
 '가벼워지다',
 '가볍다',
 '가보',
 '가보다',
 '가봣는데',
 '가분',
 '가비',
 '가빠지다',
 '가뿐하다',
 '가쁘',
 '가사',
 '가산',
 '가상',
 '가설',
 '가성',
 '가세',
 '가소롭

In [ ]:
len(res)
# stop_words = set(stop_words)
# stop_words
old_res = res

1425

In [ ]:
def remove_stopwords(res):
    return [[word for word in doc if word not in stop_words] for doc in res]
new_res = remove_stopwords(res)
new_res

TypeError: 'NoneType' object is not iterable

In [86]:
id2word = corpora.Dictionary(res)

[['탈모',
  '살',
  '시작',
  '하다',
  '약',
  '먹다',
  '모나드',
  '정',
  '피',
  '나스',
  '테',
  '리드',
  '처방',
  '받다',
  '효과',
  '언제',
  '쯤',
  '보다',
  '있다',
  '효과',
  '무조건',
  '있다',
  '가요',
  '하루하루',
  '너무',
  '걱정',
  '드네',
  '불안하다',
  '계속',
  '정수리',
  '사진',
  '찍다',
  '확인',
  '있다'],
 ['처음',
  '처방',
  '받다',
  '질문',
  '남기다',
  '처방전',
  '나시',
  '정',
  '개월',
  '정도',
  '적당하다',
  '가격',
  '처음',
  '병원',
  '가다',
  '약',
  '처방',
  '받다',
  '잘',
  '모르다',
  '글',
  '남기다',
  '병원',
  '가깝다',
  '피부',
  '가다',
  '계속',
  '이',
  '병원',
  '다니다',
  '되다',
  '가요'],
 ['하남',
  '워',
  '스튜디오',
  '하남',
  '미사',
  '점',
  '두피',
  '문신',
  '후기',
  '이다',
  '대다',
  '정수리',
  '부분',
  '탈모',
  '시작',
  '되다',
  '대다',
  '중후',
  '반',
  '부터',
  '흑',
  '채',
  '뿌리다',
  '시작',
  '하다',
  '대다',
  '초',
  '프로',
  '페',
  '복용',
  '하다',
  '개월',
  '이상',
  '복용',
  '하',
  '확실하다',
  '효과',
  '있다',
  '값',
  '부담',
  '되다',
  '끊다',
  '다시',
  '탈모',
  '진행',
  '값',
  '싸다',
  '아보',
  '모나',
  '사용',
  '해봤다',
  '확실하다',
  '프로',
  '페',
  '시아',
  '부작용',
  '있다',
  '듯',
  '느

In [202]:
texts = res
texts

[['탈모',
  '살',
  '시작',
  '약',
  '먹다',
  '모나드',
  '정',
  '피',
  '나스',
  '테',
  '리드',
  '처방',
  '받다',
  '효과',
  '언제',
  '쯤',
  '보다',
  '효과',
  '무조건',
  '가요',
  '하루하루',
  '너무',
  '걱정',
  '드네',
  '불안하다',
  '계속',
  '정수리',
  '사진',
  '찍다',
  '확인'],
 ['처음',
  '처방',
  '받다',
  '질문',
  '남기다',
  '처방전',
  '나시',
  '정',
  '개월',
  '정도',
  '적당하다',
  '가격',
  '처음',
  '병원',
  '가다',
  '약',
  '처방',
  '받다',
  '잘',
  '모르다',
  '글',
  '남기다',
  '병원',
  '가깝다',
  '피부',
  '가다',
  '계속',
  '이',
  '병원',
  '다니다',
  '되다',
  '가요'],
 ['하남',
  '워',
  '스튜디오',
  '하남',
  '미사',
  '점',
  '두피',
  '문신',
  '후기',
  '이다',
  '대다',
  '정수리',
  '부분',
  '탈모',
  '시작',
  '되다',
  '대다',
  '중후',
  '반',
  '부터',
  '흑',
  '채',
  '뿌리다',
  '시작',
  '대다',
  '초',
  '프로',
  '페',
  '복용',
  '개월',
  '이상',
  '복용',
  '하',
  '확실하다',
  '효과',
  '값',
  '부담',
  '되다',
  '끊다',
  '다시',
  '탈모',
  '진행',
  '값',
  '싸다',
  '아보',
  '모나',
  '사용',
  '해봤다',
  '확실하다',
  '프로',
  '페',
  '시아',
  '부작용',
  '듯',
  '느낌',
  '들다',
  '사다',
  '먹다',
  '못',
  '그렇게',
  '대다',
  '후반',
  '탈

In [48]:
corpus = [id2word.doc2bow(text) for text in texts]

In [49]:
corpus[:1][0]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 1),
 (14, 1),
 (15, 1),
 (16, 1),
 (17, 1),
 (18, 3),
 (19, 1),
 (20, 1),
 (21, 1),
 (22, 1),
 (23, 1),
 (24, 1),
 (25, 1),
 (26, 1),
 (27, 1),
 (28, 1),
 (29, 1),
 (30, 2)]

In [56]:
num_topics = 2
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.054*"하다" + 0.017*"있다" + 0.015*"탈모" + 0.013*"머리" + 0.013*"약" + 0.011*"되다" '
  '+ 0.010*"같다" + 0.010*"먹다" + 0.009*"이다" + 0.008*"보다"'),
 (1,
  '0.035*"하다" + 0.026*"탈모" + 0.018*"있다" + 0.017*"약" + 0.015*"먹다" + 0.013*"머리" '
  '+ 0.011*"같다" + 0.011*"되다" + 0.010*"받다" + 0.009*"보다"')]


In [57]:
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared

PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
0      0.007121  0.0       1        1  50.585708
1     -0.007121  0.0       2        1  49.414292, topic_info=    Term           Freq          Total Category  logprob  loglift
27    하다  143985.000000  143985.000000  Default  30.0000  30.0000
24    탈모   65523.000000   65523.000000  Default  29.0000  29.0000
20   정수리   16066.000000   16066.000000  Default  28.0000  28.0000
10    받다   24675.000000   24675.000000  Default  27.0000  27.0000
187    자    9322.000000    9322.000000  Default  26.0000  26.0000
..   ...            ...            ...      ...      ...      ...
179    더    7082.116866   14409.764289   Topic2  -5.4127  -0.0054
188    저    6756.704096   13494.452991   Topic2  -5.4597   0.0132
374    안    7342.881168   15773.676415   Topic2  -5.3765  -0.0597
219  아니다    7344.900000   16260.412106   Topic2  -5.3763  -0.0898
241   많이    7102.253420   15667.213207   Topic2  -5.4099  -0.0862

[217 rows x 6 columns], token_table=       Topic      Freq Term
term                       
559        1  0.337794  가르마
559        2  0.662350  가르마
1078       1  0.770565   가발
1078       2  0.229355   가발
0          1  0.389847   가요
...      ...       ...  ...
3633       2  0.725506   회정
30         1  0.456887   효과
30         2  0.543071   효과
12177      1  0.145184    흙
12177      2  0.798510    흙

[322 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2])

In [58]:
num_topics = 3
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.048*"하다" + 0.021*"탈모" + 0.020*"있다" + 0.020*"약" + 0.016*"먹다" + 0.012*"되다" '
  '+ 0.011*"같다" + 0.011*"머리" + 0.009*"보다" + 0.008*"없다"'),
 (1,
  '0.042*"하다" + 0.022*"머리" + 0.019*"탈모" + 0.015*"있다" + 0.012*"빠지다" + '
  '0.011*"같다" + 0.010*"이다" + 0.009*"되다" + 0.009*"없다" + 0.008*"이식"'),
 (2,
  '0.041*"하다" + 0.021*"탈모" + 0.015*"받다" + 0.013*"있다" + 0.011*"이식" + 0.011*"되다" '
  '+ 0.010*"같다" + 0.010*"모발" + 0.009*"먹다" + 0.009*"이다"')]


In [59]:
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.005266 -0.012936       1        1  45.774762
1     -0.016388  0.002754       2        1  33.444465
2      0.011122  0.010182       3        1  20.780772, topic_info=    Term           Freq          Total Category  logprob  loglift
10    받다   25415.000000   25415.000000  Default  30.0000  30.0000
185   이식   22309.000000   22309.000000  Default  29.0000  29.0000
82    머리   44881.000000   44881.000000  Default  28.0000  28.0000
27    하다  142678.000000  142678.000000  Default  27.0000  27.0000
181   모발   20872.000000   20872.000000  Default  26.0000  26.0000
..   ...            ...            ...      ...      ...      ...
82    머리    5074.639916   44881.765168   Topic3  -4.8798  -0.6086
45    정도    3125.661349   16762.893819   Topic3  -5.3644  -0.1084
219  아니다    2970.151979   16021.121233   Topic3  -5.4154  -0.1142
374    안    2854.436905   15995.891807   Topic3  -5.4552  -0.1523
241   많이    2826.600489   16070.258220   Topic3  -5.4650  -0.1668

[316 rows x 6 columns], token_table=       Topic      Freq Term
term                       
33         1  0.492707   가다
33         2  0.234266   가다
33         3  0.273144   가다
1078       1  0.239746   가발
1078       2  0.383906   가발
...      ...       ...  ...
11179      1  0.810778   히터
11179      2  0.135130   히터
11179      3  0.067565   히터
8416       1  0.136326   힘껏
8416       2  0.817957   힘껏

[638 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3])

In [146]:
num_topics = 5
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics,
                                       iterations=300)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]


[(0,
  '0.055*"하다" + 0.020*"있다" + 0.017*"먹다" + 0.011*"약" + 0.011*"받다" + 0.010*"탈모" '
  '+ 0.009*"같다" + 0.009*"병원" + 0.008*"복용" + 0.008*"보다"'),
 (1,
  '0.035*"하다" + 0.028*"탈모" + 0.026*"있다" + 0.016*"약" + 0.014*"먹다" + 0.012*"되다" '
  '+ 0.010*"보다" + 0.010*"이다" + 0.008*"복용" + 0.008*"같다"'),
 (2,
  '0.039*"하다" + 0.021*"약" + 0.015*"먹다" + 0.015*"같다" + 0.014*"있다" + 0.014*"탈모" '
  '+ 0.013*"머리" + 0.010*"되다" + 0.010*"빠지다" + 0.009*"보다"'),
 (3,
  '0.043*"하다" + 0.023*"탈모" + 0.021*"받다" + 0.013*"있다" + 0.013*"이식" + 0.012*"모발" '
  '+ 0.012*"병원" + 0.011*"되다" + 0.011*"약" + 0.008*"이다"'),
 (4,
  '0.052*"하다" + 0.024*"탈모" + 0.019*"머리" + 0.017*"있다" + 0.012*"약" + 0.012*"되다" '
  '+ 0.011*"같다" + 0.010*"먹다" + 0.010*"없다" + 0.009*"빠지다"')]


In [108]:
lda_model.print_topic(0)

'0.057*"하다" + 0.022*"탈모" + 0.017*"있다" + 0.016*"약" + 0.014*"먹다" + 0.012*"되다" + 0.012*"머리" + 0.010*"받다" + 0.010*"없다" + 0.008*"복용"'

In [148]:
pyLDAvis.enable_notebook()
vis5 = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
pyLDAvis.display(vis5)

# LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))
# # # this is a bit time consuming - make the if statement True
# # # if you want to execute visualization prep yourself
# if 1 == 1:
#     LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
#     with open(LDAvis_data_filepath, 'wb') as f:
#         pickle.dump(LDAvis_prepared, f)
# # load the pre-prepared pyLDAvis data from disk
# with open(LDAvis_data_filepath, 'rb') as f:
#     LDAvis_prepared = pickle.load(f)
# pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_'+ str(num_topics) +'.html')
# LDAvis_prepared

In [175]:
num_topics = 6
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics,
                                       iterations = 300)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda6 = lda_model[corpus]

[(0,
  '0.060*"하다" + 0.021*"있다" + 0.019*"탈모" + 0.015*"약" + 0.014*"먹다" + 0.014*"복용" '
  '+ 0.010*"머리" + 0.010*"없다" + 0.010*"같다" + 0.010*"되다"'),
 (1,
  '0.042*"하다" + 0.030*"약" + 0.022*"먹다" + 0.022*"탈모" + 0.015*"있다" + 0.015*"받다" '
  '+ 0.011*"같다" + 0.010*"병원" + 0.009*"되다" + 0.009*"머리"'),
 (2,
  '0.043*"하다" + 0.024*"있다" + 0.017*"탈모" + 0.014*"이식" + 0.014*"먹다" + 0.013*"모발" '
  '+ 0.011*"약" + 0.011*"되다" + 0.011*"머리" + 0.010*"이다"'),
 (3,
  '0.031*"탈모" + 0.025*"하다" + 0.017*"있다" + 0.017*"약" + 0.015*"되다" + 0.013*"먹다" '
  '+ 0.012*"보다" + 0.012*"머리" + 0.012*"없다" + 0.010*"같다"'),
 (4,
  '0.043*"하다" + 0.022*"있다" + 0.021*"탈모" + 0.015*"약" + 0.014*"같다" + 0.013*"되다" '
  '+ 0.012*"것" + 0.010*"보다" + 0.010*"없다" + 0.009*"머리"'),
 (5,
  '0.050*"하다" + 0.021*"머리" + 0.017*"탈모" + 0.013*"있다" + 0.012*"같다" + '
  '0.011*"빠지다" + 0.010*"되다" + 0.009*"보다" + 0.008*"먹다" + 0.008*"약"')]


In [177]:
def get_topic(doc_lda):
    for i, doc in enumerate(doc_lda):
        # print(f"Document {i}:")
        topics = sorted(doc, key=lambda x: x[1], reverse=True)  # Sort topics by probability
        yield topics[0][0]
        # for topic_id, prob in topics:
    #     print(f"Topic {topic_id}: {prob:.4f}")

topics = [topic for topic in get_topic(doc_lda6)]
topics


[3,
 1,
 3,
 5,
 5,
 5,
 2,
 2,
 4,
 5,
 4,
 4,
 4,
 5,
 5,
 0,
 1,
 3,
 4,
 3,
 5,
 1,
 1,
 3,
 0,
 5,
 0,
 5,
 5,
 1,
 5,
 4,
 0,
 3,
 1,
 3,
 3,
 5,
 2,
 1,
 5,
 4,
 0,
 1,
 5,
 2,
 5,
 3,
 0,
 0,
 3,
 1,
 1,
 5,
 2,
 5,
 1,
 5,
 4,
 3,
 4,
 3,
 0,
 1,
 5,
 4,
 3,
 4,
 5,
 5,
 3,
 1,
 3,
 1,
 2,
 3,
 2,
 1,
 4,
 5,
 4,
 5,
 1,
 1,
 3,
 1,
 3,
 5,
 3,
 5,
 2,
 3,
 3,
 1,
 4,
 4,
 1,
 0,
 1,
 5,
 5,
 4,
 2,
 5,
 4,
 0,
 4,
 4,
 1,
 0,
 1,
 5,
 5,
 5,
 0,
 0,
 0,
 1,
 5,
 2,
 5,
 5,
 1,
 0,
 1,
 5,
 3,
 2,
 4,
 5,
 1,
 5,
 2,
 4,
 2,
 3,
 0,
 4,
 1,
 1,
 3,
 5,
 3,
 4,
 1,
 0,
 5,
 0,
 3,
 5,
 4,
 1,
 1,
 2,
 3,
 1,
 4,
 1,
 5,
 5,
 1,
 5,
 3,
 1,
 2,
 2,
 5,
 3,
 4,
 3,
 1,
 1,
 0,
 0,
 1,
 1,
 3,
 1,
 5,
 2,
 5,
 0,
 3,
 1,
 2,
 5,
 1,
 0,
 3,
 0,
 5,
 5,
 4,
 0,
 5,
 1,
 5,
 0,
 5,
 5,
 5,
 2,
 2,
 4,
 3,
 5,
 5,
 1,
 1,
 5,
 1,
 2,
 5,
 4,
 1,
 1,
 1,
 3,
 1,
 0,
 5,
 3,
 5,
 3,
 0,
 1,
 5,
 1,
 2,
 1,
 3,
 5,
 5,
 3,
 5,
 1,
 5,
 1,
 2,
 2,
 5,
 3,
 5,
 5,
 3,
 1,
 0,
 3,
 1,
 1,


In [122]:
len(topics)

54005

In [123]:
print(len(tm))
tm.head()
tm['Topics'] = topics

54005


In [180]:
# tm['Topic'] = topics
# tm = tm.drop('Topic',axis=1)
tm.head()

,Title,DateTime,Author,Rank,Category,Text,Images,Views,Href,title_text,Topic
0,탈모 3년째,2023-09-01 13:01:06,뷔놈,5,Fuck탈모,21살부터 시작해서 3년지나고나서야 약을 먹습니다 모나드정 피나스테리드 처방 받았는...,NaN,74.0,https://daedamo.com/story/1871888?overlaps=1,탈모 3년째 21살부터 시작해서 3년지나고나서야 약을 먹습니다 모나드정 피나스테리드...,3
1,처음 처방받아서 질문남겨요,2023-09-01 12:08:44,K27135528642303…,3,고민상담,처방전 16500원\r\n모나시아정 63000원인데 3개월에 이정도면 적당한 가격인...,NaN,91.0,https://daedamo.com/story/1871881?overlaps=1,처음 처방받아서 질문남겨요 처방전 16500원\r\n모나시아정 63000원인데 3개...,1
2,하남 아워스튜디오하남미사점 두피문신 후기 입니다.,2023-09-01 11:53:37,모닝98,2,Fuck탈모,20대부터 정수리 부분 탈모가 시작되 20대 중후반 부터 흑채 뿌리기 시작했습니다....,NaN,112.0,https://daedamo.com/story/1871880?overlaps=1,하남 아워스튜디오하남미사점 두피문신 후기 입니다. 20대부터 정수리 부분 탈모가 시...,3
3,여의도 탈모성지다녀온후기씁니다,2023-09-01 11:21:25,K29993214812309…,2,Fuck탈모,회사 스트레스로 아침저녁으로. 머리빠지는듯해서\r\n\r\n걱정이였는데\r\n\r\...,NaN,168.0,https://daedamo.com/story/1871873?overlaps=1,여의도 탈모성지다녀온후기씁니다 회사 스트레스로 아침저녁으로. 머리빠지는듯해서\r\n...,5
4,M자 모발이식 질문,2023-09-01 11:17:19,빛바랜머리,2,Fuck탈모,모발에 힘도 없고 가늘고 M자까지 있네요\r\n스트레스 받아서 수술을 받을까 생각중...,NaN,142.0,https://daedamo.com/story/1871872?overlaps=1,M자 모발이식 질문 모발에 힘도 없고 가늘고 M자까지 있네요\r\n스트레스 받아서 ...,5


In [142]:
for i in range(0,6):
    pd.DataFrame(tm[tm['Topic'] == i]['Text'],columns=['Text']).to_csv(f'text{i}.csv',index=False,encoding='utf-8-sig')

In [141]:
df = pd.read_csv('text0.csv')
df

,Text
0,처방전 16500원\r\n모나시아정 63000원인데 3개월에 이정도면 적당한 가격인...
1,가족력 부모님다 70대 60대신데도 풍성하시고 친가외가 탈모이신분이 한명도없습니다\...
2,20살에 처음으로 속알머리가 없는 거 알았고..\r\n약 꾸준히 먹다 끊다를 반복\...
3,5년동안 사귄 여친과 이별했습니다\r\n\r\n바람에 휘날려 헝크러진 머리를 다급히...
4,23살부터 군전역 후 바로 복용하기 시작했어요\r\n벌써 9년차 복용중이네요\r\n...
...,...
13131,우연찮게 시간이 좀 나서 15년 다닌 탈모병원에 상담했어요\r\n\r\n병원가서 제...
13132,그럼 하루 약은 언제 드시고\r\n\r\n영양제는 언제쯤 드시나요??
13133,결혼 안하는게 나을거같네요 심해서\r\n괜히 나중에 아들한테 미안할듯 혼자 솰아야겠...
13134,몇달전부터 머리가 많이 가렵고 하더니만 이렇게 되어있네요... ㅠㅠ\r\n\r\n일...


In [176]:
pyLDAvis.enable_notebook()
vis6 = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
pyLDAvis.display(vis6)

# LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))
# # # this is a bit time consuming - make the if statement True
# # # if you want to execute visualization prep yourself
# if 1 == 1:
#     LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
#     with open(LDAvis_data_filepath, 'wb') as f:
#         pickle.dump(LDAvis_prepared, f)
# # load the pre-prepared pyLDAvis data from disk
# with open(LDAvis_data_filepath, 'rb') as f:
#     LDAvis_prepared = pickle.load(f)
# pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_'+ str(num_topics) +'.html')
# LDAvis_prepared

In [186]:
tm[tm['Topic'] == 2]['Href']

6             https://daedamo.com/story/1871864?overlaps=1
7             https://daedamo.com/story/1871862?overlaps=1
38       https://daedamo.com/story/1871641?page=2&overl...
45       https://daedamo.com/story/1871565?page=2&overl...
54       https://daedamo.com/story/1871514?page=2&overl...
                               ...                        
53967    https://daedamo.com/story/1438525?page=1499&ov...
53984    https://daedamo.com/story/1438396?page=1500&ov...
53990    https://daedamo.com/story/1438353?page=1500&ov...
53998    https://daedamo.com/story/1438300?page=1500&ov...
54003    https://daedamo.com/story/1438251?page=1500&ov...
Name: Href, Length: 6856, dtype: object

In [173]:
num_topics = 7
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics,
                                       iterations = 300)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.037*"하다" + 0.022*"약" + 0.017*"있다" + 0.017*"먹다" + 0.016*"탈모" + 0.013*"되다" '
  '+ 0.012*"머리" + 0.011*"같다" + 0.009*"복용" + 0.009*"정도"'),
 (1,
  '0.038*"하다" + 0.019*"있다" + 0.015*"아보" + 0.015*"되다" + 0.014*"다트" + 0.014*"먹다" '
  '+ 0.013*"프로" + 0.011*"보다" + 0.011*"받다" + 0.010*"들다"'),
 (2,
  '0.034*"하다" + 0.029*"탈모" + 0.022*"먹다" + 0.022*"약" + 0.020*"있다" + 0.015*"머리" '
  '+ 0.014*"같다" + 0.012*"되다" + 0.010*"이다" + 0.010*"정수리"'),
 (3,
  '0.062*"하다" + 0.015*"탈모" + 0.015*"있다" + 0.014*"이식" + 0.011*"모발" + 0.011*"되다" '
  '+ 0.010*"머리" + 0.010*"받다" + 0.009*"같다" + 0.009*"보다"'),
 (4,
  '0.044*"하다" + 0.020*"약" + 0.018*"있다" + 0.012*"먹다" + 0.012*"없다" + 0.011*"탈모" '
  '+ 0.009*"병원" + 0.009*"되다" + 0.008*"받다" + 0.008*"복용"'),
 (5,
  '0.042*"하다" + 0.026*"머리" + 0.019*"탈모" + 0.017*"빠지다" + 0.014*"있다" + 0.014*"약" '
  '+ 0.013*"먹다" + 0.011*"같다" + 0.009*"보다" + 0.009*"때"'),
 (6,
  '0.042*"하다" + 0.028*"탈모" + 0.025*"있다" + 0.012*"되다" + 0.011*"두피" + 0.011*"들" '
  '+ 0.009*"머리" + 0.009*"들다" + 0.009*"같다" + 0.008*"약"'

In [174]:
pyLDAvis.enable_notebook()
vis7 = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
pyLDAvis.display(vis7)

# LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))
# # # this is a bit time consuming - make the if statement True
# # # if you want to execute visualization prep yourself
# if 1 == 1:
#     LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
#     with open(LDAvis_data_filepath, 'wb') as f:
#         pickle.dump(LDAvis_prepared, f)
# # load the pre-prepared pyLDAvis data from disk
# with open(LDAvis_data_filepath, 'rb') as f:
#     LDAvis_prepared = pickle.load(f)
# pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_'+ str(num_topics) +'.html')
# LDAvis_prepared

In [103]:
num_topics = 8
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics,
                                       iterations=300)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.035*"하다" + 0.018*"받다" + 0.018*"약" + 0.018*"탈모" + 0.017*"머리" + 0.015*"있다" '
  '+ 0.014*"같다" + 0.014*"빠지다" + 0.012*"이다" + 0.011*"병원"'),
 (1,
  '0.027*"하다" + 0.021*"먹다" + 0.013*"있다" + 0.012*"약" + 0.012*"없다" + '
  '0.011*"미녹시딜" + 0.011*"같다" + 0.011*"바르다" + 0.010*"보다" + 0.009*"되다"'),
 (2,
  '0.024*"하다" + 0.023*"약" + 0.022*"먹다" + 0.022*"있다" + 0.017*"프로" + 0.017*"머리" '
  '+ 0.016*"시아" + 0.016*"탈모" + 0.015*"페" + 0.012*"복용"'),
 (3,
  '0.060*"하다" + 0.016*"있다" + 0.015*"탈모" + 0.014*"먹다" + 0.013*"약" + 0.010*"머리" '
  '+ 0.010*"같다" + 0.010*"되다" + 0.010*"복용" + 0.008*"없다"'),
 (4,
  '0.043*"하다" + 0.036*"탈모" + 0.021*"약" + 0.018*"머리" + 0.016*"있다" + 0.013*"먹다" '
  '+ 0.010*"되다" + 0.010*"보다" + 0.009*"같다" + 0.009*"이다"'),
 (5,
  '0.034*"하다" + 0.019*"같다" + 0.019*"탈모" + 0.018*"머리" + 0.016*"있다" + 0.015*"먹다" '
  '+ 0.013*"약" + 0.011*"보다" + 0.010*"없다" + 0.009*"빠지다"'),
 (6,
  '0.045*"하다" + 0.026*"있다" + 0.018*"되다" + 0.017*"이식" + 0.015*"모발" + 0.014*"탈모" '
  '+ 0.010*"같다" + 0.009*"보다" + 0.009*"병원" + 0.009*"약

In [104]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis
import os
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.002472 -0.003401       1        1  24.549346
4     -0.007353 -0.015299       2        1  23.651861
6     -0.024777  0.000523       3        1  16.231386
0      0.009280 -0.024914       4        1  12.675682
2      0.059884 -0.014854       5        1   9.266511
1      0.011563  0.064216       6        1   4.723838
7     -0.050134 -0.005456       7        1   4.528891
5      0.004008 -0.000815       8        1   4.372485, topic_info=    Term           Freq          Total Category  logprob  loglift
27    하다  138025.000000  138025.000000  Default  30.0000  30.0000
7     먹다   41038.000000   41038.000000  Default  29.0000  29.0000
82    머리   43490.000000   43490.000000  Default  28.0000  28.0000
24    탈모   64902.000000   64902.000000  Default  27.0000  27.0000
193   같다   34623.000000   34623.000000  Default  26.0000  26.0000
..   ...            ...            ...      ...      ...      ...
219  아니다     612.582243   16196.274345   Topic8  -5.4354  -0.1450
59   그렇다     540.937302    8592.828802   Topic8  -5.5598   0.3645
214   많다     553.635219    9879.213505   Topic8  -5.5366   0.2482
94    복용     594.431620   22051.209486   Topic8  -5.4655  -0.4837
10    받다     574.366682   25080.929619   Topic8  -5.4998  -0.6468

[756 rows x 6 columns], token_table=       Topic      Freq Term
term                       
31         1  0.140002   가격
31         2  0.071564   가격
31         3  0.347748   가격
31         4  0.116379   가격
31         5  0.145213   가격
...      ...       ...  ...
15638      5  0.200888   흑발
15638      7  0.803554   흑발
12177      1  0.074647    흙
12177      2  0.821119    흙
8416       4  0.907572   힘껏

[2214 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 5, 7, 1, 3, 2, 8, 6])

In [66]:
num_topics = 9
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.043*"탈모" + 0.032*"하다" + 0.020*"있다" + 0.015*"머리" + 0.012*"같다" + 0.010*"되다" '
  '+ 0.010*"이다" + 0.009*"정수리" + 0.009*"약" + 0.009*"보다"'),
 (1,
  '0.045*"하다" + 0.025*"있다" + 0.016*"탈모" + 0.016*"들" + 0.010*"먹다" + 0.010*"같다" '
  '+ 0.009*"약" + 0.009*"보다" + 0.009*"되다" + 0.009*"없다"'),
 (2,
  '0.045*"하다" + 0.023*"머리" + 0.018*"있다" + 0.016*"먹다" + 0.015*"같다" + 0.013*"약" '
  '+ 0.012*"탈모" + 0.012*"빠지다" + 0.011*"되다" + 0.009*"보다"'),
 (3,
  '0.054*"하다" + 0.020*"약" + 0.016*"빠지다" + 0.016*"먹다" + 0.015*"탈모" + 0.013*"머리" '
  '+ 0.013*"있다" + 0.011*"같다" + 0.010*"보다" + 0.010*"이다"'),
 (4,
  '0.059*"하다" + 0.021*"되다" + 0.014*"탈모" + 0.012*"있다" + 0.011*"같다" + 0.010*"약" '
  '+ 0.009*"머리" + 0.008*"병원" + 0.008*"이식" + 0.008*"먹다"'),
 (5,
  '0.031*"하다" + 0.023*"약" + 0.023*"있다" + 0.020*"탈모" + 0.018*"먹다" + 0.017*"되다" '
  '+ 0.010*"모발" + 0.010*"같다" + 0.009*"이식" + 0.008*"많이"'),
 (6,
  '0.044*"하다" + 0.027*"있다" + 0.015*"복용" + 0.014*"약" + 0.013*"먹다" + 0.011*"탈모" '
  '+ 0.009*"머리" + 0.008*"되다" + 0.008*"이다" + 0.008*"받다"'

In [67]:
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.016986 -0.017630       1        1  16.352485
0     -0.039980  0.001745       2        1  16.349608
3     -0.003697 -0.000246       3        1  12.758069
7     -0.001762  0.000891       4        1  12.458727
8      0.001489  0.024444       5        1  11.403090
6      0.006318 -0.004695       6        1   9.870385
1      0.025232 -0.023026       7        1   8.111372
5      0.018140  0.019868       8        1   7.173841
4      0.011245 -0.001352       9        1   5.522421, topic_info=    Term           Freq          Total Category  logprob  loglift
27    하다  139469.000000  139469.000000  Default  30.0000  30.0000
39    되다   35532.000000   35532.000000  Default  29.0000  29.0000
185   이식   18630.000000   18630.000000  Default  28.0000  28.0000
24    탈모   69286.000000   69286.000000  Default  27.0000  27.0000
16     약   44947.000000   44947.000000  Default  26.0000  26.0000
..   ...            ...            ...      ...      ...      ...
117   없다    1095.921538   25138.804808   Topic9  -5.0873  -0.2365
11    보다    1091.629887   27608.777239   Topic9  -5.0912  -0.3341
94    복용     912.218390   21851.088133   Topic9  -5.2707  -0.2798
189    중     780.153733   13386.194941   Topic9  -5.4271   0.0539
209    것     752.737463   18989.573404   Topic9  -5.4629  -0.3316

[800 rows x 6 columns], token_table=      Topic      Freq Term
term                      
31        1  0.094998   가격
31        2  0.080665   가격
31        3  0.064332   가격
31        4  0.100664   가격
31        5  0.123997   가격
...     ...       ...  ...
1242      6  0.059364   흡연
1242      7  0.056396   흡연
1242      8  0.053427   흡연
1242      9  0.249328   흡연
8416      2  0.831500   힘껏

[2522 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 4, 8, 9, 7, 2, 6, 5])

In [52]:
num_topics = 10
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.060*"하다" + 0.021*"탈모" + 0.015*"머리" + 0.014*"있다" + 0.014*"약" + 0.011*"이다" '
  '+ 0.011*"복용" + 0.010*"받다" + 0.010*"되다" + 0.010*"먹다"'),
 (1,
  '0.058*"하다" + 0.026*"있다" + 0.022*"탈모" + 0.011*"들" + 0.010*"보다" + 0.009*"들다" '
  '+ 0.008*"없다" + 0.008*"같다" + 0.008*"되다" + 0.008*"것"'),
 (2,
  '0.034*"약" + 0.028*"하다" + 0.024*"있다" + 0.020*"탈모" + 0.020*"먹다" + 0.015*"되다" '
  '+ 0.010*"보다" + 0.009*"복용" + 0.009*"같다" + 0.008*"머리"'),
 (3,
  '0.037*"하다" + 0.024*"있다" + 0.021*"탈모" + 0.015*"되다" + 0.013*"먹다" + 0.013*"같다" '
  '+ 0.013*"이식" + 0.012*"모발" + 0.012*"약" + 0.010*"것"'),
 (4,
  '0.043*"하다" + 0.023*"먹다" + 0.020*"약" + 0.015*"되다" + 0.015*"있다" + 0.014*"같다" '
  '+ 0.013*"탈모" + 0.011*"보다" + 0.010*"머리" + 0.009*"받다"'),
 (5,
  '0.038*"탈모" + 0.034*"머리" + 0.019*"하다" + 0.014*"정수리" + 0.013*"빠지다" + '
  '0.012*"있다" + 0.012*"이다" + 0.011*"보다" + 0.011*"같다" + 0.009*"약"'),
 (6,
  '0.053*"하다" + 0.016*"있다" + 0.015*"머리" + 0.012*"같다" + 0.012*"되다" + 0.011*"먹다" '
  '+ 0.010*"이식" + 0.010*"탈모" + 0.009*"빠지다" + 0.009*"약"')

In [55]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis
import os
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.014991  0.022965       1        1  14.045222
5     -0.027378 -0.044790       2        1  13.619448
9     -0.026122  0.006125       3        1  10.602500
1      0.023824 -0.001682       4        1  10.254616
6     -0.000029 -0.001966       5        1  10.252850
8      0.060842 -0.003270       6        1   9.864683
0     -0.002916 -0.003572       7        1   9.141809
3      0.009610  0.000639       8        1   8.595343
2     -0.017518  0.029814       9        1   8.468946
7     -0.005323 -0.004262      10        1   5.154584, topic_info=    Term           Freq          Total Category  logprob  loglift
27    하다  137730.000000  137730.000000  Default  30.0000  30.0000
16     약   45861.000000   45861.000000  Default  29.0000  29.0000
24    탈모   67924.000000   67924.000000  Default  28.0000  28.0000
82    머리   43553.000000   43553.000000  Default  27.0000  27.0000
7     먹다   37904.000000   37904.000000  Default  26.0000  26.0000
..   ...            ...            ...      ...      ...      ...
78     때     808.169846   16298.807746  Topic10  -5.3229  -0.0388
30    효과     735.532697   13563.973815  Topic10  -5.4171   0.0507
219  아니다     778.998505   16653.401528  Topic10  -5.3597  -0.0971
188    저     711.538593   13403.356031  Topic10  -5.4502   0.0295
185   이식     705.026297   19094.820403  Topic10  -5.4594  -0.3337

[891 rows x 6 columns], token_table=       Topic      Freq   Term
term                         
757        1  0.106190  가늘어지다
757        2  0.168134  가늘어지다
757        3  0.082171  가늘어지다
757        4  0.048038  가늘어지다
757        5  0.080906  가늘어지다
...      ...       ...    ...
7352       9  0.093051   흉측하다
12177      1  0.099392      흙
12177      2  0.795136      흙
12177      5  0.049696      흙
8416       5  0.783470     힘껏

[3037 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 6, 10, 2, 7, 9, 1, 4, 3, 8])

In [36]:
for index in top_words:
    print(cv.get_feature_names_out()[index])

후
좀
탈모
비
들
고민
좋다
보다
같다
곳
절개
상담
되다
수술
받다
있다
병원
모발
이식
하다


In [96]:
import pyLDAvis.gensim_models
import pyLDAvis.lda_model
from gensim import corpora


In [95]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda5, dtm, cv)
pyLDAvis.display(vis)

AttributeError: 'CountVectorizer' object has no attribute 'token2id'